# DeLaN Network Inference 

Following script demonstrates to get DeLaN rearrangement Sequence from Random pick and place points on table top environment.

install these two required libaries for current session : ( run the next cell)

In [6]:
!pip install ortools
!pip install pandas

     |████████████████████████████████| 9.5 MB 4.7 MB/s eta 0:00:01


In [7]:
import matplotlib.pyplot as plt
import numpy as np
import time
import read_data_network
#from ortools import constraint_solver
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from dataset_custom_z import TrajectoryDataset_customScipy
from torch.utils.data import DataLoader
import sys
import pickle

def merge_nodes(starts, goals, depot=None):
    nodes = []
    if depot:
        nodes.append(depot)
    for start in starts:
        nodes.append(start)
    for goal in goals:
        nodes.append(goal)
    return nodes

def create_data_model_euclidean(N, nodes):
    """Stores the data for the problem."""
    data = {}
    from scipy.spatial import distance_matrix
    dm = distance_matrix(nodes, nodes)
    #read_data_network.pprint(dm)
    data['distance_matrix'] = dm


    data['pickups_deliveries'] = []
    data['demands'] = [0]

    for i in range(N):
        data['pickups_deliveries'].append([i+1, i+1+N])
    for i in range(N):
        data['demands'].append(1)
    for i in range(N):
        data['demands'].append(-1)

    data['num_vehicles'] = 1
    data['depot'] = 0

    data['vehicle_capacities'] = [1]
    # print("cost_eucledian inside")
    # print(data)
    return data

def create_data_model_joint(N, nodes, network='delan'):
    """Stores the data for the problem."""
    data = {}
    dm = read_data_network.get_joint_distance_matrix(nodes, network)

    data['distance_matrix'] = dm
    data['pickups_deliveries'] = []
    data['demands'] = [0]

    for i in range(N):
        data['pickups_deliveries'].append([i+1, i+1+N])
    for i in range(N):
        data['demands'].append(1)
    for i in range(N):
        data['demands'].append(-1)

    data['num_vehicles'] = 1
    data['depot'] = 0

    data['vehicle_capacities'] = [1]
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    import time
    t = time.time()
    total_distance = 0
    sol = []
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Picking Sequence : \n'
        route_distance = 0
        odd = 0 
        while not routing.IsEnd(index):
            s = manager.IndexToNode(index)
            if odd !=0 and odd %2 == 1:
                plan_output += ' {} -> '.format(s)
            sol.append(s)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            
            odd += 1
            
        s = manager.IndexToNode(index)
        sol.append(s)
#         plan_output += '{}\n'.format(s)
        #plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance
    print("total time taken:", time.time()-t)
    #print('Total Distance of all routes: {}m'.format(total_distance))
    return sol

def solve(data):
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        2,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')


    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        10000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Define Transportation Requests.
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index) <=
            distance_dimension.CumulVar(delivery_index))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # search_parameters.first_solution_strategy = (
    #     routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    sol = None
    # Print solution on console.
    if solution:
        sol = print_solution(data, manager, routing, solution)
    return sol

def sample_data(N=6):
    start_xs = np.linspace(0.23, 0.73, num=50).tolist()
    start_ys = np.linspace(-0.3, 0.3, num=50).tolist()
    goal_xs = np.linspace(0.23, 0.73, num=50).tolist()
    goal_ys = np.linspace(-0.3, 0.3, num=50).tolist()

    start_zs = np.linspace(0.1, 0.3, num=50).tolist()
    goal_zs = np.linspace(0.1, 0.3, num=50).tolist()
    from random import sample
    depot = (0.24605024, -0.22180356,  0.41969074)
#     N = int(sys.argv[1])
    start_x = sample(start_xs, N)
    start_y = sample(start_ys, N)
    goal_x = sample(goal_xs, N)
    goal_y = sample(goal_ys, N)
    start_z = sample(start_zs,N)
    goal_z = sample(goal_zs, N)
    starts = []
    goals = []
    for i in range(N):
        starts.append((start_x[i], start_y[i],0.1))
    for i in range(N):
        goals.append((goal_x[i], goal_y[i], 0.1))
    return depot, starts, goals

def sample_data_training(N=6):
    TRAJ_train = TrajectoryDataset_customScipy()
    trainloader = DataLoader(TRAJ_train, batch_size=1, drop_last=True, shuffle=True)
    depot = (0.24605024, -0.22180356,  0.41969074)
#     N = int(sys.argv[1])
    i = 0
    starts = []
    goals = []
    for x, y, net_input, cost, start_joint, end_joint in trainloader:
        traj = []
        device='cpu'
        x = x.to(device).numpy()
        y = y.to(device).numpy()
        print("X = ", x)
        starts.append((x[0][0], x[0][1], x[0][2]))
        goals.append((y[0][0],y[0][1], y[0][2]))
        i+=1
        if i>=N:
            break
    return depot, starts, goals


In [8]:
def run(n):
    # N represents the number of objects to rearrange
    N =n
    depot, starts, goals = sample_data(N=N)
    z = 0.0
#     N = len(starts)

    print("Number of Objects : " + str(N))


    print("Random Start Points: ")
    print(starts)
    print()
    print("Random Goal Points: ")
    print(goals)
    nodes = merge_nodes(starts, goals, depot)
    data_e = create_data_model_euclidean(N, nodes)
    data_j = create_data_model_joint(N, nodes, network='delan')
    data_j_nn = create_data_model_joint(N, nodes, network='fnn')
    print()
    print("Solving in Euclidean Space")

    start_time = time.time()
    sol_e = solve(data_e)
    total_time = time.time() - start_time

    points = [depot] + starts + goals
    route_index = []
    for so in sol_e:
        route_index.append(list(points[int(so)]))


        
    print("Solving in DeLAN Space")
    start_time = time.time()
    sol_j = solve(data_j)
    total_time = time.time()-start_time
    
    print("Solving in NN Space")
    start_time = time.time()
    sol_j = solve(data_j_nn)
    total_time = time.time()-start_time   
    

    route_index = []
    for so in sol_j:
        route_index.append(list(points[int(so)]))

    
run(10)

Number of Objects : 10
Random Start Points: 
[(0.5259183673469388, 0.15306122448979592, 0.1), (0.3320408163265306, -0.07959183673469389, 0.1), (0.30142857142857143, -0.030612244897959162, 0.1), (0.29122448979591836, -0.0673469387755102, 0.1), (0.2810204081632653, 0.22653061224489796, 0.1), (0.23, -0.16530612244897958, 0.1), (0.2606122448979592, -0.14081632653061224, 0.1), (0.5463265306122449, -0.10408163265306122, 0.1), (0.7197959183673469, 0.16530612244897958, 0.1), (0.3626530612244898, 0.04285714285714287, 0.1)]

Random Goal Points: 
[(0.46469387755102043, 0.18979591836734694, 0.1), (0.6687755102040817, 0.25102040816326526, 0.1), (0.3728571428571429, 0.01836734693877551, 0.1), (0.2810204081632653, 0.07959183673469389, 0.1), (0.5259183673469388, 0.2755102040816327, 0.1), (0.4851020408163266, 0.20204081632653065, 0.1), (0.6483673469387755, 0.2877551020408163, 0.1), (0.6177551020408163, -0.01836734693877551, 0.1), (0.2504081632653061, 0.21428571428571425, 0.1), (0.49530612244897954, -0.